In [1]:
from bs4 import BeautifulSoup
import requests
from lxml import etree
import csv

In [2]:
headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }

In [3]:
def get_info(url, writer):
    req = requests.get(url, headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    dom = etree.HTML(str(soup))
    
    #Variabili
    global name_txt
    global c_id_txt
    global c_type_txt
    global address_txt
    global status_txt
    global registered_txt
    global incorporated_txt
    global dissolved_txt
    
    #Company name
    name = dom.xpath('//p[@class = "heading-xlarge"]')
    if name:
        name_txt = name[0].text
    else:
        name_txt = None
    
    #Company number
    c_id = dom.xpath('//p[@id = "company-number"]/strong')
    if c_id:
        c_id_txt = c_id[0].text
    else:
        c_id_txt = None
    
    #Company type
    c_type = dom.xpath('//dt[text() = "Company type"]/../dd')
    if c_type:
        c_type_txt = c_type[0].text
        c_type_txt = c_type_txt.replace(' ', '')
        c_type_txt = c_type_txt.replace('\n', '')
    else:
        c_type_txt = None
    
    #Registered office address
    if(c_type_txt == 'Overseasentity'):
        address = dom.xpath('//dt[text() = "Correspondence address"]/../dd')
        if address:
            address_txt = address[0].text
            address_txt = address_txt.replace('\n', '')
            address_txt = address_txt.replace(' ', '')
            address_txt = address_txt.replace(',', ' ')
        else:
            address_txt = None
    else:
        address = dom.xpath('//dt[text() = "Registered office address"]/../dd')
        if address:
            address_txt = address[0].text
            address_txt = address_txt.replace('\n', '')
            address_txt = address_txt.replace(' ', '')
            address_txt = address_txt.replace(',', ' ')
        else:
            address_txt = None
        
    #Company status
    status = dom.xpath('//dt[text() = "Company status"]/../dd')
    if status:
        status_txt = status[0].text 
        status_txt = status_txt.replace(' ', '')
        status_txt = status_txt.replace('\n', '')
    else:
        status_txt = None
    
    #Registration Date
    if(status_txt == 'Registered'):
        registered = dom.xpath('//dt[text() = "Registered on"]/../dd')
        incorporated_txt = None
        if registered:
            registered_txt = registered[0].text
        else:
            registered_txt = None
    else:
        incorporated = dom.xpath('//dt[text() = "Incorporated on"]/../dd')
        registered_txt = None
        if incorporated:
            incorporated_txt = incorporated[0].text
    #Dissolved on
    if(status_txt == 'Dissolved'):
        dissolved = dom.xpath('//dt[text() = "Dissolved on"]/../dd')
        if dissolved:
            dissolved_txt = dissolved[0].text
        else:
            dissolved_txt = None
    else:
        dissolved_txt = None
    
    #Save Line
    line = [name_txt, c_id_txt, status_txt, c_type_txt, registered_txt, incorporated_txt, dissolved_txt, address_txt]
    writer.writerow(line)

In [4]:
b = 0
url = 'https://find-and-update.company-information.service.gov.uk'
ds = open('govuk-dataset.csv', 'w')
head = ['Name', 'Company ID', 'Company Status', 'Company Type', 
        'Registration Date', 'Incorporation Date', 'Dissolution Date', 
       'Office Address']
writer = csv.writer(ds)
writer.writerow(head)
#RICERCA SU 'a'
for x in range(1, 200):
    url2 = url + '/search/companies?q=a&page=' + str(x)
    req = requests.get(url2, headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    for a in soup.find_all('a', class_ = 'govuk-link', href = True):
        if '/company' in a['href']:
            b += 1
            get_info(url + a['href'], writer)

#RICERCA SU 't'        
for x in range(1, 200):
    url2 = url + '/search/companies?q=t&page=' + str(x)
    req = requests.get(url2, headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    for a in soup.find_all('a', class_ = 'govuk-link', href = True):
        if '/company' in a['href']:
            b += 1
            get_info(url + a['href'], writer)
ds.close()
print(b)

0
